In [1]:
import numpy as np
from PIL import Image

In [2]:
target_bits = [7, 6, 5, 4, 3, 2, 1]

In [3]:
image = Image.open("tesla_8_bit.png")
print(f"Görüntü modu: {image.mode}")
if image.mode == '1':
    print("Görüntü 1 bit.")
elif image.mode in ['L', 'P']:
    print("Görüntü 8 bit.")
elif image.mode == 'RGB':
    print("Görüntü 24 bit.")
elif image.mode == 'RGBA':
    print("Görüntü 32 bit.")
else:
    print("Diğer bit derinlikleri.")

Görüntü modu: L
Görüntü 8 bit.


In [4]:
def quantize(image, original_bits, target_bits):
    """
    Kuantalama işlemi.
    Formül:  floor((Vi * (V'max - V'min) / (Vmax - Vmin)))

    image: PIL.Image.Image
    bits: int
    """
    np_image = np.array(image)
    Vnew_max = 2 ** target_bits - 1
    Vnew_min = 0
    Vmax = 2 ** original_bits - 1
    Vmin = 0
    quantized_image = np.floor((np_image * ((Vnew_max - Vnew_min) / (Vmax - Vmin))))
    return quantized_image

In [5]:
def de_quantize(image, original_bits, target_bits):
    """
    De-kuantalama işlemi.
    Formül:  floor((Vi * (Vmax - Vmin) / (V'max - V'min)))

    image: PIL.Image.Image
    bits: int
    """
    np_image = np.array(image)
    Vnew_max = 2 ** target_bits - 1
    Vnew_min = 0
    Vmax = 2 ** original_bits - 1
    Vmin = 0
    de_quantized_image = np.floor((np_image * ((Vmax - Vmin) / (Vnew_max - Vnew_min))))
    return de_quantized_image

In [7]:
def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [8]:
def mae(original, compressed):
    mae = np.mean(np.abs(original - compressed))
    return mae

In [8]:
def apply_metrics(image1, image2):
    """
    PSNR ve MAE hesaplayan fonksiyon.
    """
    image1 = np.array(image1)
    image2 = np.array(image2)
    mse = np.mean((image1 - image2) ** 2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse))
    mae = np.mean(np.abs(image1 - image2))
    return psnr, mae

In [ ]:
for target_bit in target_bits:
    quantized_image = quantize(image, 8, target_bit)
    de_quantized_image = de_quantize(quantized_image, 8, target_bit)
    print(f"{target_bit} bit için PSNR değeri: {psnr(np.array(image), de_quantized_image)}")
    print(f"{target_bit} bit için MAE değeri: {mae(np.array(image), de_quantized_image)}")
    de_quantized_image = Image.fromarray(de_quantized_image.astype(np.uint8))
    print(f"{target_bit} bit için PSNR ve MAE değerleri: {apply_metrics(image, de_quantized_image)}")
    de_quantized_image.save(f"tesla_{target_bit}_bit_dequantized.png")